In [2]:
#imports
import sys
sys.path.append('../')
import numpy as np
import pandas as pd
import scipy as sp
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from helpers.products import Products
from directed_graph_generators.directed_tree import DirectedTree
from directed_graph_generators.lattice import DirectedLattice
from directed_graph_generators.directed_stochastic_block_model import DirectedStochasticBlockModel
from directed_graph_generators.directed_erdos_renyi import DirectedErdosRenyi

from clustering_algorithms.exponential_clustering import Exponential_Clustering, Exponential_Clustering_no_evecs
from clustering_algorithms.zanetti_clustering import DSBM_Clustering_Zanetti

from helpers.get_hermitian_adjacency_matrix import get_hermitian_adjacency_matrix
from helpers.deduce_metagraph import deduce_metagraph

## First and Second Eigenvector Plots
We will construct a graph with the following adjancency matrix structure:
$$
\begin{pmatrix}
G(n,p) & G(n,q) & 0 & G(n,q) & 0 & 0 \\
0 & G(n,p) & G(n,q) & 0 & 0 & 0 \\
G(n,q) & 0 & G(n,p) & 0 & 0 & 0 \\
0 & 0 & 0 & G(n,p) & G(n,q) & 0 \\
0 & 0 & 0 & 0 & G(n,p) & G(n,q) \\
0 & 0 & 0 & G(n,q) & 0 & G(n,p) \\
\end{pmatrix}
$$

(Two 3 cycles joined together with one edge.)

In [12]:
graph_description = '2 3cycles'

In [13]:
#We write a quick function for constructing a 4 cycle (not a DSBM) network.
def construct_graph_2_3_cycles(n,p,q):
    # n cluster size. graph will have 4n vertices
    # p probability of within cluster edge
    # q probability of between cluster edge
    # returns hermitian adjacency matrix
    A = np.zeros((6*n,6*n))
    # filling diagonals with directed G(n,p)s
    for i in range(6):
        block = np.random.binomial(1, p, int((n)*(n)))
        block_row, block_col = i, i
        A[n*block_row:n*(block_row+1), n*block_col:n*(block_col+1)] = block.reshape(n,n)
    # fill in (1,2), (1,4), (2,3), (3,1), (4,5), (5,6), (6,4)
    A[n*0:n*1, n*1:n*2] = np.random.binomial(1, q, int((n)*(n))).reshape(n,n)
    A[n*0:n*1, n*3:n*4] = np.random.binomial(1, q, int((n)*(n))).reshape(n,n)
    A[n*1:n*2, n*2:n*3] = np.random.binomial(1, q, int((n)*(n))).reshape(n,n)
    A[n*2:n*3, n*0:n*1] = np.random.binomial(1, q, int((n)*(n))).reshape(n,n)
    A[n*3:n*4, n*4:n*5] = np.random.binomial(1, q, int((n)*(n))).reshape(n,n)
    A[n*4:n*5, n*5:n*6] = np.random.binomial(1, q, int((n)*(n))).reshape(n,n)
    A[n*5:n*6, n*3:n*4] = np.random.binomial(1, q, int((n)*(n))).reshape(n,n)
    return A


Here we just run through a quick example with p = q = 0.1 and n = 100 (so 600 nodes total). We do this just to get a feel for how the plots should look and make sure everything works as expected.

In [14]:
# lets create a 4 cycle network with n = 200, p = 0.1, q = 0.1
n = 100
p = 0.1
q = 0.1
adjacency_matrix = construct_graph_2_3_cycles(n,p,q)
# Generate hermitian adjacency matrix
A = get_hermitian_adjacency_matrix(adjacency_matrix, normalize=True)
# first look at eigenvalues of normalized hermitian laplacian
eig_vals, eig_vecs = np.linalg.eig(np.eye(6*n) - A)
#sorting according to largest in magnitude
eig_vals = np.real(eig_vals)
idx = np.abs(eig_vals).argsort()
eig_vals = eig_vals[idx]
eig_vecs = eig_vecs[:,idx]

In [15]:
# plot eig_vecs[:,0] and eig_vecs[:,1] separately
# define clusters for colouring
clusters = ['cluster 1']*n + ['cluster 2']*n + ['cluster 3']*n + ['cluster 4']*n + ['cluster 5']*n + ['cluster 6']*n
# x and y coordinates (real and imag parts of first eigenvector)
x_0 = np.real(eig_vecs[:,0])
y_0 = np.imag(eig_vecs[:,0])
fig = go.Figure()
fig = px.scatter(x=x_0, y=y_0, color=clusters)
# add title and label axis and legend
fig.update_layout(title=f'First Eigenvector of Normalized Hermitian Laplacian ({graph_description}, n = {n}, p = {p}, q = {q})', xaxis_title='Real', yaxis_title='Imaginary', legend_title='Cluster')
fig.show()

In [16]:
# repeating for the second eigenvector
x_1 = np.real(eig_vecs[:,1])
y_1 = np.imag(eig_vecs[:,1])
fig = go.Figure()
fig = px.scatter(x=x_1, y=y_1, color=clusters)
# add title and label axis and legend
fig.update_layout(title='First Eigenvector of Normalized Hermitian Laplacian (4 cycle, n= 200, p = 0.1, q = 0.1)', xaxis_title='Real', yaxis_title='Imaginary', legend_title='Cluster')
fig.show()

Lets repeat with p = 1, q = 0.25.

In [18]:
# We will repeat with p set to 1 so there is no randomness.
p = 1
q = 0.25
adjacency_matrix = construct_graph_2_3_cycles(n,p,q)
# Generate hermitian adjacency matrix
A = get_hermitian_adjacency_matrix(adjacency_matrix, normalize=True)
# first look at eigenvalues of normalized hermitian laplacian
determined_eig_vals, determined_eig_vecs = np.linalg.eig(np.eye(6*n) - A)
#sorting according to largest in magnitude
determined_eig_vals = np.real(determined_eig_vals)
idx = np.abs(determined_eig_vals).argsort()
determined_eig_vals = determined_eig_vals[idx]
determined_eig_vecs = determined_eig_vecs[:,idx]

det_x_0 = np.real(determined_eig_vecs[:,0])
det_y_0 = np.imag(determined_eig_vecs[:,0])
fig = go.Figure()
fig = px.scatter(x=det_x_0, y=det_y_0, color=clusters)
# add title and label axis and legend
fig.update_layout(title=f'First Eigenvector of Normalized Hermitian Laplacian ({graph_description}, n = {n}, p = {p}, q = {q})', xaxis_title='Real', yaxis_title='Imaginary', legend_title='Cluster')
fig.show()

# repeating for the second eigenvector
det_x_1 = np.real(determined_eig_vecs[:,1])
det_y_1 = np.imag(determined_eig_vecs[:,1])
fig = go.Figure()
fig = px.scatter(x=det_x_1, y=det_y_1, color=clusters)
# add title and label axis and legend
fig.update_layout(title=f'Second Eigenvector of Normalized Hermitian Laplacian ({graph_description}, n = {n}, p = {p}, q = {q})', xaxis_title='Real', yaxis_title='Imaginary', legend_title='Cluster')
fig.show()

det_x_2 = np.real(determined_eig_vecs[:,2])
det_y_2 = np.imag(determined_eig_vecs[:,2])
fig = go.Figure()
fig = px.scatter(x=det_x_2, y=det_y_2, color=clusters)
# add title and label axis and legend
fig.update_layout(title=f'Third Eigenvector of Normalized Hermitian Laplacian ({graph_description}, n = {n}, p = {p}, q = {q})', xaxis_title='Real', yaxis_title='Imaginary', legend_title='Cluster')
fig.show()

This is as expected for the first eigenvector. In computing i(A-A^T), all the edges within the clusters 'cancel out' so in minimizing
$$ x^* \mathcal{L} x = \frac{\sum_{u \rightarrow v} |x_u - i x_v| }{\sum d_u |x_u|^2} $$
x can just take $x = D^{1/2} \chi$ where $\chi$ is the indicator vector taking values 1, i, -1 and -i.
For the second eigenvector, we see that its entries still stick the axes but the clusters overlap.

As expected, the first eigenvalue is 0. See below:

In [19]:
print('First eigenvalue: ', determined_eig_vals[0])
print('Second eigenvalue: ', determined_eig_vals[1])
print('Third eigenvalue: ', determined_eig_vals[2])

First eigenvalue:  0.11100236537770795
Second eigenvalue:  0.3042457418077569
Third eigenvalue:  0.7737732019703949


In [21]:
# plot determined_eig_vals
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(5*n), y=determined_eig_vals, mode='markers', name='eigenvalues'))

# plot deduced metagraph eigenvalues
M = deduce_metagraph(A,clusters = [[i for i in range(0,n)],[i for i in range(n,2*n)],[i for i in range(2*n,3*n)],[i for i in range(3*n,4*n)]],normalize = True)
meta_eig_vals, meta_eig_vecs = np.linalg.eig(M)
meta_eig_vals = np.real(meta_eig_vals)
idx = np.abs(meta_eig_vals).argsort()
meta_eig_vals = meta_eig_vals[idx]

fig.add_trace(go.Scatter(x=np.arange(4), y=meta_eig_vals, mode='markers', name='metagraph eigenvalues'))
fig.update_layout(title=f'Eigenvalues of Normalized Hermitian Laplacian (4 cycle, n = 200, p = {p}, q = {q})', xaxis_title='Index', yaxis_title='Eigenvalue')
fig.show()

Making q smaller (q=0.04) but still with p = 1, since the degrees become more varied, we have more spread along the axes.

In [26]:
# We will repeat with p set to 1 so there is no randomness.
p = 1
q = 1
adjacency_matrix = construct_graph_2_3_cycles(n,p,q)
# Generate hermitian adjacency matrix
A = get_hermitian_adjacency_matrix(adjacency_matrix, normalize=True)
# first look at eigenvalues of normalized hermitian laplacian
determined_eig_vals, determined_eig_vecs = np.linalg.eig(np.eye(6*n) - A)
#sorting according to largest in magnitude
determined_eig_vals = np.real(determined_eig_vals)
idx = np.abs(determined_eig_vals).argsort()
determined_eig_vals = determined_eig_vals[idx]
determined_eig_vecs = determined_eig_vecs[:,idx]

det_x_0 = np.real(determined_eig_vecs[:,0])
det_y_0 = np.imag(determined_eig_vecs[:,0])
fig = go.Figure()
fig = px.scatter(x=det_x_0, y=det_y_0, color=clusters)
# add title and label axis and legend
fig.update_layout(title=f'First Eigenvector of Normalized Hermitian Laplacian (4 cycle, n = 200, p = {p}, q = {q})', xaxis_title='Real', yaxis_title='Imaginary', legend_title='Cluster')
fig.show()

# repeating for the second eigenvector
det_x_1 = np.real(determined_eig_vecs[:,1])
det_y_1 = np.imag(determined_eig_vecs[:,1])
fig = go.Figure()
fig = px.scatter(x=det_x_1, y=det_y_1, color=clusters)
# add title and label axis and legend
fig.update_layout(title=f'First Eigenvector of Normalized Hermitian Laplacian (4 cycle, n = 200, p = {p}, q = {q})', xaxis_title='Real', yaxis_title='Imaginary', legend_title='Cluster')
fig.show()

det_x_2 = np.real(determined_eig_vecs[:,2])
det_y_2 = np.imag(determined_eig_vecs[:,2])
fig = go.Figure()
fig = px.scatter(x=det_x_2, y=det_y_2, color=clusters)
# add title and label axis and legend
fig.update_layout(title=f'Third Eigenvector of Normalized Hermitian Laplacian (4 cycle, n = 200, p = {p}, q = {q})', xaxis_title='Real', yaxis_title='Imaginary', legend_title='Cluster')
fig.show()

det_x_3 = np.real(determined_eig_vecs[:,3])
det_y_3 = np.imag(determined_eig_vecs[:,3])
fig = go.Figure()
fig = px.scatter(x=det_x_3, y=det_y_3, color=clusters)
# add title and label axis and legend
fig.update_layout(title=f'Fourth Eigenvector of Normalized Hermitian Laplacian (4 cycle, n = 200, p = {p}, q = {q})', xaxis_title='Real', yaxis_title='Imaginary', legend_title='Cluster')
fig.show()


Now, when p > q but p is small as well, we get less pronounced clusters.

In [11]:
# We will repeat with p set to 1 so there is no randomness.
p = 0.2
q = 0.04
adjacency_matrix = construct_graph2(n,p,q)
# Generate hermitian adjacency matrix
A = get_hermitian_adjacency_matrix(adjacency_matrix, normalize=True)
# first look at eigenvalues of normalized hermitian laplacian
determined_eig_vals, determined_eig_vecs = np.linalg.eig(np.eye(4*n) - A)
#sorting according to largest in magnitude
determined_eig_vals = np.real(determined_eig_vals)
idx = np.abs(determined_eig_vals).argsort()
determined_eig_vals = determined_eig_vals[idx]
determined_eig_vecs = determined_eig_vecs[:,idx]

det_x_0 = np.real(determined_eig_vecs[:,0])
det_y_0 = np.imag(determined_eig_vecs[:,0])
fig = go.Figure()
fig = px.scatter(x=det_x_0, y=det_y_0, color=clusters)
# add title and label axis and legend
fig.update_layout(title=f'First Eigenvector of Normalized Hermitian Laplacian (4 cycle, n = 200, p = {p}, q = {q})', xaxis_title='Real', yaxis_title='Imaginary', legend_title='Cluster')
fig.show()

# repeating for the second eigenvector
det_x_1 = np.real(determined_eig_vecs[:,1])
det_y_1 = np.imag(determined_eig_vecs[:,1])
fig = go.Figure()
fig = px.scatter(x=det_x_1, y=det_y_1, color=clusters)
# add title and label axis and legend
fig.update_layout(title=f'First Eigenvector of Normalized Hermitian Laplacian (4 cycle, n = 200, p = {p}, q = {q})', xaxis_title='Real', yaxis_title='Imaginary', legend_title='Cluster')
fig.show()